<a href="https://colab.research.google.com/github/trismegisto78/romeo/blob/master/testBidirLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/trismegisto78/romeo

fatal: destination path 'romeo' already exists and is not an empty directory.


In [14]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [15]:
with open('/gdrive/my-drive/strMacd2004.csv', 'r') as f:
  letto=  f.read()
!cat /gdrive/strMacd2004.csv

FileNotFoundError: ignored

In [10]:
righe_file = open('/gdrive/strMacd2004.csv',encoding='utf-8',errors='ignore').read().split('\n')

import numpy as np
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

FileNotFoundError: ignored

In [0]:
# define problem properties
n_timesteps = 200
# define LSTM
model = Sequential()
model.add(Bidirectional(LSTM(n_timesteps, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

In [0]:


samplePos=0

# create a sequence classification instance
def get_sequence():
  X = np.zeros(n_timesteps)
  Y = np.zeros(n_timesteps)
  for k in range(0,n_timesteps):
    value=float(righe_file[samplePos+k])/3000
    value2=float(righe_file[samplePos+k+1])/3000
    #print(value,' ',value2)
    X[k]=value
    if value2<value:
      Y[k]=1
  
  X = X.reshape(1, n_timesteps, 1)
  y = Y.reshape(1, n_timesteps, 1)
  #
  return X, y


# train LSTM
for epoch in range(2000):
  X,y = get_sequence()
  samplePos=samplePos+1
  model.fit(X, y, epochs=1, batch_size=1, verbose=2)
  if samplePos>4000000 :
    samplePos=0
# evaluate LSTM
X,y = get_sequence()

yhat = model.predict_classes(X, verbose=0)
for i in range(n_timesteps):
	print('Expected:', y[0, i], 'Predicted', yhat[0, i])